In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import euclidean
from xgboost import XGBRegressor
import joblib
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score

os.environ["OMP_NUM_THREADS"] = "1"

df = pd.read_csv("train.csv")
df = df.drop(['data'], axis=1)
df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])

#df.isnull().sum() #verifying that the data is consistent
df#df.describe()

,source_name,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time,...,postcode_y,city_x,city_y,county_x,county_y,Sstate,Dstate,loc_formatted_x,loc_formatted_y,haversine_distance
0,Anand_VUNagar_DC (Gujarat),Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,10.435660,14.0,11.0,11.9653,14.0,...,388600.0,Anand,Khambhat,Anand City Taluka,Khambhat Taluka,Gujarat,Gujarat,"Anand, GJ, India","Khambhat, GJ, India",43.973301
1,Anand_VUNagar_DC (Gujarat),Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,18.936842,24.0,20.0,21.7243,10.0,...,388600.0,Anand,Khambhat,Anand City Taluka,Khambhat Taluka,Gujarat,Gujarat,"Anand, GJ, India","Khambhat, GJ, India",43.973301
2,Anand_VUNagar_DC (Gujarat),Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,27.637279,40.0,28.0,32.5395,16.0,...,388600.0,Anand,Khambhat,Anand City Taluka,Khambhat Taluka,Gujarat,Gujarat,"Anand, GJ, India","Khambhat, GJ, India",43.973301
3,Anand_VUNagar_DC (Gujarat),Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,36.118028,62.0,40.0,45.5620,21.0,...,388600.0,Anand,Khambhat,Anand City Taluka,Khambhat Taluka,Gujarat,Gujarat,"Anand, GJ, India","Khambhat, GJ, India",43.973301
4,Anand_VUNagar_DC (Gujarat),Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,39.386040,68.0,44.0,54.2181,6.0,...,388600.0,Anand,Khambhat,Anand City Taluka,Khambhat Taluka,Gujarat,Gujarat,"Anand, GJ, India","Khambhat, GJ, India",43.973301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104853,Sonipat_Kundli_H (Haryana),Gurgaon_Bilaspur_HB (Haryana),2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,45.258278,94.0,60.0,67.9280,12.0,...,NaN,Kundli,Gurgaon,Sonipat,NaN,Haryana,Haryana,"Kundli, HR, India","Gurgaon, HR, India",50.395739
104854,Sonipat_Kundli_H (Haryana),Gurgaon_Bilaspur_HB (Haryana),2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,54.092531,120.0,76.0,85.6829,26.0,...,NaN,Kundli,Gurgaon,Sonipat,NaN,Haryana,Haryana,"Kundli, HR, India","Gurgaon, HR, India",50.395739
104855,Sonipat_Kundli_H (Haryana),Gurgaon_Bilaspur_HB (Haryana),2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,66.163591,140.0,88.0,97.0933,20.0,...,NaN,Kundli,Gurgaon,Sonipat,NaN,Haryana,Haryana,"Kundli, HR, India","Gurgaon, HR, India",50.395739
104856,Sonipat_Kundli_H (Haryana),Gurgaon_Bilaspur_HB (Haryana),2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,73.680667,158.0,98.0,111.2709,17.0,...,NaN,Kundli,Gurgaon,Sonipat,NaN,Haryana,Haryana,"Kundli, HR, India","Gurgaon, HR, India",50.395739


In [5]:
# xgboost
print("Training XGBoost model ...")
features = ['osrm_time','haversine_distance','start_scan_to_end_scan', 'osrm_distance', 'segment_actual_time','segment_osrm_time','segment_osrm_distance']
X = df[features]
y = df[['actual_distance_to_destination','actual_time']]
model = XGBRegressor(random_state=108)  # Enable GPU support
model.fit(X, y)
print("XGBoost model training completed")
#joblib.dump(model, 'route-optmiz.joblib')

#train calc
xtrp = model.predict(X)
trmse = np.sqrt(mean_squared_error(y, xtrp))
tr2 = r2_score(y, xtrp)
#testing
td = pd.read_csv("test.csv")
X_te = td[features]
y_te = td[['actual_distance_to_destination','actual_time']]
y_pred = model.predict(X_te)
rmse = np.sqrt(mean_squared_error(y_te, y_pred))
r2 = r2_score(y_te, y_pred)
joblib.dump(model, 'route-optmiz.joblib')
print(f"Model Evaluation:\nRMSE: {rmse:.10f}\nR² Score: {r2:.10f}\n\n")
print(f"Train RMSE: {trmse:.4f}, Test RMSE: {rmse:.4f}")
print(f"Train R²: {tr2:.4f}, Test R²: {r2:.4f}")

Training XGBoost model ...
XGBoost model training completed
Model Evaluation:
RMSE: 56.3990776257
R² Score: 0.9914014339


Train RMSE: 31.3308, Test RMSE: 56.3991
Train R²: 0.9969, Test R²: 0.9914


In [7]:
'''
df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['date'] = df['od_start_time'].dt.date
grouped = df.groupby(['source_name', 'date'])
from sklearn.cluster import KMeans
for (source_name, date), group in grouped:
    source = {'name': source_name, 'lat': group['source_lat'].iloc[0], 'long': group['source_long'].iloc[0]}
    destinations = group[['destination_name', 'destination_lat', 'destination_long']].to_dict('records')
    k = 2  # Number of vehicles; adjust as needed
    if len(destinations) < k:
        print(f"Skipping {source_name} on {date}: too few deliveries")
        continue
    coords = [(d['destination_lat'], d['destination_long']) for d in destinations]
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(coords)
    for i, d in enumerate(destinations):
        d['cluster'] = labels[i]
        
from haversine import haversine, Unit
all_points = [source] + cluster_points
num_locations = len(all_points)
dist_matrix = [[haversine((all_points[i]['lat'], all_points[i]['long']),
                         (all_points[j]['lat'], all_points[j]['long']),
                         unit=Unit.KILOMETERS) * 1000  # Convert to meters
                for j in range(num_locations)]
               for i in range(num_locations)]
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
manager = pywrapcp.RoutingIndexManager(num_locations, 1, 0)  # 1 vehicle, depot at 0
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(dist_matrix[from_node][to_node])
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
solution = routing.SolveWithParameters(search_parameters)'''

'from sklearn.cluster import KMeans\nfor (source_name, date), group in grouped:\n    source = {\'name\': source_name, \'lat\': group[\'source_lat\'].iloc[0], \'long\': group[\'source_long\'].iloc[0]}\n    destinations = group[[\'destination_name\', \'destination_lat\', \'destination_long\']].to_dict(\'records\')\n    k = 2  # Number of vehicles; adjust as needed\n    if len(destinations) < k:\n        print(f"Skipping {source_name} on {date}: too few deliveries")\n        continue\n    coords = [(d[\'destination_lat\'], d[\'destination_long\']) for d in destinations]\n    kmeans = KMeans(n_clusters=k, random_state=42)\n    labels = kmeans.fit_predict(coords)\n    for i, d in enumerate(destinations):\n        d[\'cluster\'] = labels[i]\n        \nfrom haversine import haversine, Unit\nall_points = [source] + cluster_points\nnum_locations = len(all_points)\ndist_matrix = [[haversine((all_points[i][\'lat\'], all_points[i][\'long\']),\n                         (all_points[j][\'lat\'], all_

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from haversine import haversine, Unit
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import folium
import re

# Function to sanitize file names
def sanitize_filename(name):
    return re.sub(r'[^a-zA-Z0-9_-]', '_', name)

# Assuming df is loaded with necessary columns
df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['date'] = df['od_start_time'].dt.date
grouped = df.groupby(['source_name', 'date'])

def compute_distance_matrix(points):
    n = len(points)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = haversine(
                (points[i]['lat'], points[i]['long']),
                (points[j]['lat'], points[j]['long']),
                unit=Unit.KILOMETERS
            ) * 1000  # Convert to meters for OR-Tools
    return dist_matrix

def solve_vrp_for_cluster(cluster_id, cluster_points, source):
    all_points = [source] + cluster_points
    num_locations = len(all_points)
    
    # Compute distance matrix
    dist_matrix = compute_distance_matrix(all_points)
    
    # Create routing manager (1 vehicle, depot at source = index 0)
    manager = pywrapcp.RoutingIndexManager(num_locations, 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    # Define distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(dist_matrix[from_node][to_node])
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Set search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Solve VRP
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print(f"\nOptimal Route for Cluster {cluster_id} from {source['name']}:")
        index = routing.Start(0)
        route = []
        route_coords = []
        total_distance = 0
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(all_points[node]['name'])
            route_coords.append((all_points[node]['lat'], all_points[node]['long']))
            next_index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(next_index):
                total_distance += dist_matrix[node][manager.IndexToNode(next_index)]
            index = next_index
        # Return to source
        node = manager.IndexToNode(index)
        route.append(all_points[node]['name'])
        route_coords.append((all_points[node]['lat'], all_points[node]['long']))
        print(" -> ".join(route))
        print(f"Total Distance: {total_distance / 1000:.2f} km")
        return route_coords, total_distance / 1000
    else:
        print(f"No solution found for Cluster {cluster_id}")
        return None, None

# Process each source-date group
for (source_name, date), group in grouped:
    # Extract source coordinates
    source = {
        'name': source_name,
        'lat': group['source_lat'].iloc[0],
        'long': group['source_long'].iloc[0]
    }
    
    # Extract unique destinations
    destinations = group[['destination_name', 'destination_lat', 'destination_long']].rename(
        columns={
            'destination_name': 'name',
            'destination_lat': 'lat',
            'destination_long': 'long'
        }
    ).drop_duplicates(subset=['lat', 'long']).to_dict('records')
    
    num_destinations = len(destinations)
    if num_destinations < 1:
        print(f"Skipping {source_name} on {date}: no deliveries")
        continue
    
    # Set number of clusters
    if num_destinations == 1:
        k = 1
        for d in destinations:
            d['cluster'] = 0
    else:
        k = min(4, num_destinations)
        coords = [(d['lat'], d['long']) for d in destinations]
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(coords)
        for i, d in enumerate(destinations):
            d['cluster'] = labels[i]
    
    # Colors for routes (used consistently across maps)
    colors = ['green', 'purple', 'orange', 'darkblue']
    
    # Process each cluster and create a separate map
    for cluster_id in range(k):
        cluster_points = [d for d in destinations if d['cluster'] == cluster_id]
        if not cluster_points:
            continue  # Unlikely with KMeans, but safeguard
        
        # Create a new map for this cluster
        m = folium.Map(location=[source['lat'], source['long']], zoom_start=10)
        
        # Add source marker
        folium.Marker(
            [source['lat'], source['long']],
            popup=source['name'],
            icon=folium.Icon(color='red', icon='home')
        ).add_to(m)
        
        # Add destination markers for this cluster only
        for dest in cluster_points:
            folium.Marker(
                [dest['lat'], dest['long']],
                popup=dest['name'],
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(m)
        
        # Solve VRP and plot route
        route_coords, total_distance = solve_vrp_for_cluster(cluster_id, cluster_points, source)
        if route_coords:
            folium.PolyLine(
                route_coords,
                color=colors[cluster_id % len(colors)],
                weight=2.5,
                opacity=1,
                popup=f"Cluster {cluster_id}: {total_distance:.2f} km"
            ).add_to(m)
        
        # Save the map
        sanitized_source_name = sanitize_filename(source_name)
        map_file = f"./route-map/route_map_{sanitized_source_name}_{date.strftime('%Y-%m-%d')}_cluster_{cluster_id}.html"
        m.save(map_file)
        print(f"Map saved as {map_file}")

load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\ortools\.libs\ortools.dll...

Optimal Route for Cluster 0 from AMD_Memnagar (Gujarat):
AMD_Memnagar (Gujarat) -> Ahmedabad_East_H_1 (Gujarat) -> AMD_Memnagar (Gujarat)
Total Distance: 105.22 km
Map saved as ./route-map/route_map_AMD_Memnagar__Gujarat__2018-09-12_cluster_0.html

Optimal Route for Cluster 0 from AMD_Memnagar (Gujarat):
AMD_Memnagar (Gujarat) -> Ahmedabad_East_H_1 (Gujarat) -> AMD_Memnagar (

C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



Optimal Route for Cluster 0 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> Anand_Vaghasi_IP (Gujarat) -> Surat_HUB (Gujarat) -> Godhra_DC (Gujarat) -> Amd_Chandkheda_Dc (Gujarat) -> Amdavad_East (Gujarat) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 531.06 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-12_cluster_0.html

Optimal Route for Cluster 1 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> Gandhidham_Sector1A_IP (Gujarat) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 253.13 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-12_cluster_1.html

Optimal Route for Cluster 2 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> Mehsana_Panchot_IP (Gujarat) -> AMD_Satellite (Gujarat) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 148.88 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-12_cluster_2.html

Optimal Route for Cluster 3 from Ahmedabad_East_

C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



Optimal Route for Cluster 0 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> AMD_Memnagar (Gujarat) -> Mehsana_Panchot_IP (Gujarat) -> Udaipur_Mangri_I (Rajasthan) -> Godhra_DC (Gujarat) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 474.80 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-16_cluster_0.html

Optimal Route for Cluster 1 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> Gurgaon_Bilaspur_HB (Haryana) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 745.59 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-16_cluster_1.html

Optimal Route for Cluster 2 from Ahmedabad_East_H_1 (Gujarat):
Ahmedabad_East_H_1 (Gujarat) -> Anand_Vaghasi_IP (Gujarat) -> Surat_HUB (Gujarat) -> Bhavnagar_ChtrGIDC_IP (Gujarat) -> Ahmedabad_East_H_1 (Gujarat)
Total Distance: 310.54 km
Map saved as ./route-map/route_map_Ahmedabad_East_H_1__Gujarat__2018-09-16_cluster_2.html

Optimal Route for Cluster 3 from Ahmedabad

...

C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMe

Map saved as ./route-map/route_map_Bangarapet_KolarRd_D__Karnataka__2018-09-20_cluster_0.html

Optimal Route for Cluster 1 from Bangarapet_KolarRd_D (Karnataka):
Bangarapet_KolarRd_D (Karnataka) -> Bangalore_East_I_20 (Karnataka) -> Bangarapet_KolarRd_D (Karnataka)
Total Distance: 296.93 km
Map saved as ./route-map/route_map_Bangarapet_KolarRd_D__Karnataka__2018-09-20_cluster_1.html

Optimal Route for Cluster 0 from Bangarapet_KolarRd_D (Karnataka):
Bangarapet_KolarRd_D (Karnataka) -> Bethamangala_FshryOFC_D (Karnataka) -> Bangarapet_KolarRd_D (Karnataka)
Total Distance: 363.01 km
Map saved as ./route-map/route_map_Bangarapet_KolarRd_D__Karnataka__2018-09-21_cluster_0.html

Optimal Route for Cluster 0 from Bangarapet_KolarRd_D (Karnataka):
Bangarapet_KolarRd_D (Karnataka) -> Bethamangala_FshryOFC_D (Karnataka) -> Bangarapet_KolarRd_D (Karnataka)
Total Distance: 363.01 km
Map saved as ./route-map/route_map_Bangarapet_KolarRd_D__Karnataka__2018-09-22_cluster_0.html

Optimal Route for Clu

C:\Users\Asus\anaconda3\envs\tf-gpu\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Map saved as ./route-map/route_map_Banka_Wardno6_D__Bihar__2018-09-14_cluster_0.html

Optimal Route for Cluster 0 from Banka_Wardno6_D (Bihar):
Banka_Wardno6_D (Bihar) -> Bhagalpur_Pbroad_DC (Bihar) -> Banka_Wardno6_D (Bihar)
Total Distance: 41.45 km
Map saved as ./route-map/route_map_Banka_Wardno6_D__Bihar__2018-09-15_cluster_0.html

Optimal Route for Cluster 0 from Banka_Wardno6_D (Bihar):
Banka_Wardno6_D (Bihar) -> Bhagalpur_Pbroad_DC (Bihar) -> Banka_Wardno6_D (Bihar)
Total Distance: 41.45 km
Map saved as ./route-map/route_map_Banka_Wardno6_D__Bihar__2018-09-16_cluster_0.html

Optimal Route for Cluster 0 from Banka_Wardno6_D (Bihar):
Banka_Wardno6_D (Bihar) -> Bhagalpur_Pbroad_DC (Bihar) -> Banka_Wardno6_D (Bihar)
Total Distance: 41.45 km
Map saved as ./route-map/route_map_Banka_Wardno6_D__Bihar__2018-09-17_cluster_0.html

Optimal Route for Cluster 0 from Banka_Wardno6_D (Bihar):
Banka_Wardno6_D (Bihar) -> Bhagalpur_Pbroad_DC (Bihar) -> Banka_Wardno6_D (Bihar)
Total Distance: 41.45

KeyboardInterrupt: 

In [ ]:
''' #plotly library
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from haversine import haversine, Unit
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import plotly.graph_objects as go
import re

# Function to sanitize file names
def sanitize_filename(name):
    return re.sub(r'[^a-zA-Z0-9_-]', '_', name)

# Assuming df is loaded with necessary columns
df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['date'] = df['od_start_time'].dt.date
grouped = df.groupby(['source_name', 'date'])
df = df.drop_duplicates(subset=['source_name', 'source_lat', 'source_long', 
                                  'destination_name', 'destination_lat', 'destination_long'])

def compute_distance_matrix(points):
    n = len(points)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = haversine(
                (points[i]['lat'], points[i]['long']),
                (points[j]['lat'], points[j]['long']),
                unit=Unit.KILOMETERS
            ) * 1000  # Convert to meters for OR-Tools
    return dist_matrix

def solve_vrp_for_cluster(cluster_id, cluster_points, source):
    all_points = [source] + cluster_points
    num_locations = len(all_points)
    
    # Compute distance matrix
    dist_matrix = compute_distance_matrix(all_points)
    
    # Create routing manager (1 vehicle, depot at source = index 0)
    manager = pywrapcp.RoutingIndexManager(num_locations, 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    # Define distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(dist_matrix[from_node][to_node])
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Set search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Solve VRP
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print(f"\nOptimal Route for Cluster {cluster_id} from {source['name']}:")
        index = routing.Start(0)
        route = []
        route_coords = []
        total_distance = 0
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(all_points[node]['name'])
            route_coords.append((all_points[node]['lat'], all_points[node]['long']))
            next_index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(next_index):
                total_distance += dist_matrix[node][manager.IndexToNode(next_index)]
            index = next_index
        # Return to source
        node = manager.IndexToNode(index)
        route.append(all_points[node]['name'])
        route_coords.append((all_points[node]['lat'], all_points[node]['long']))
        print(" -> ".join(route))
        print(f"Total Distance: {total_distance / 1000:.2f} km")
        return route_coords, total_distance / 1000
    else:
        print(f"No solution found for Cluster {cluster_id}")
        return None, None

# Process each source-date group
for (source_name, date), group in grouped:
    # Extract source coordinates
    source = {
        'name': source_name,
        'lat': group['source_lat'].iloc[0],
        'long': group['source_long'].iloc[0]
    }
    
    # Extract unique destinations
    destinations = group[['destination_name', 'destination_lat', 'destination_long']].rename(
        columns={
            'destination_name': 'name',
            'destination_lat': 'lat',
            'destination_long': 'long'
        }
    ).drop_duplicates(subset=['lat', 'long']).to_dict('records')
    
    num_destinations = len(destinations)
    if num_destinations < 1:
        print(f"Skipping {source_name} on {date}: no deliveries")
        continue
    
    # Set number of clusters
    if num_destinations == 1:
        k = 1
        for d in destinations:
            d['cluster'] = 0
    else:
        k = min(5, num_destinations)
        coords = [(d['lat'], d['long']) for d in destinations]
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(coords)
        for i, d in enumerate(destinations):
            d['cluster'] = labels[i]
    
    # Colors for routes
    colors = ['green', 'purple', 'orange', 'darkblue']
    
    # Create a single figure for all clusters
    fig = go.Figure()
    
    # Add source marker
    fig.add_trace(go.Scattergeo(
        lat=[source['lat']],
        lon=[source['long']],
        mode='markers',
        marker=dict(size=10, color='red'),
        name='Source'
    ))
    
    # Process each cluster and add to the same figure
    all_routes = []
    for cluster_id in range(k):
        cluster_points = [d for d in destinations if d['cluster'] == cluster_id]
        if not cluster_points:
            continue
        
        # Solve VRP and get route
        route_coords, total_distance = solve_vrp_for_cluster(cluster_id, cluster_points, source)
        if route_coords:
            # Extract lat and lon for the route
            lats, lons = zip(*route_coords)
            # Add route to figure
            fig.add_trace(go.Scattergeo(
                lat=lats,
                lon=lons,
                mode='lines+markers',
                line=dict(width=2, color=colors[cluster_id % len(colors)]),
                name=f'Cluster {cluster_id} Route ({total_distance:.2f} km)'
            ))
            # Add destination markers for this cluster
            cluster_lats, cluster_lons = zip(*[(d['lat'], d['long']) for d in cluster_points])
            fig.add_trace(go.Scattergeo(
                lat=list(cluster_lats),
                lon=list(cluster_lons),
                mode='markers',
                marker=dict(size=8, color='blue'),
                name=f'Cluster {cluster_id} Destinations'
            ))
    
    # Update layout
    fig.update_layout(
        title=f'Routes for {source_name} on {date}',
        geo=dict(
            showland=True,
            landcolor='rgb(217, 217, 217)',
            subunitcolor='rgb(255, 255, 255)',
            countrycolor='rgb(255, 255, 255)',
            showlakes=True,
            lakecolor='rgb(255, 255, 255)',
            showsubunits=True,
            showcountries=True,
            resolution=50,
            projection_type='mercator'
        ),
        showlegend=True
    )
    
    # Save the figure
    sanitized_source_name = sanitize_filename(source_name)
    map_file = f"./route-map/route_map_{sanitized_source_name}_{date.strftime('%Y-%m-%d')}.html"
    fig.write_html(map_file)
    print(f"Map saved as {map_file}")
    '''